In [1]:
from nltk import FreqDist
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
import nltk
from nltk.corpus import wordnet
from nltk import FreqDist
from nltk.tokenize import TweetTokenizer
from nltk.tag import pos_tag
import re, string
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import time
import random
import numpy as np
import json
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pandas as pd
import scipy.io

In [2]:


def Probabilistic_Method2(unique_words): 
    h = 2.2*len(unique_words)/(math.log(len(unique_words)))
    return  math.ceil(h)
    
def Random_Resolving_Set(unique_words):
    h = Probabilistic_Method2(unique_words)
    dictionary = dict()
    unique_words_poppy = unique_words
    for i in range(1,len(unique_words_poppy)):
        dictionary[i] = unique_words_poppy.pop()
    r = []
    q = []
    print(len(dictionary))
    for i in range(1, h):
        Rlen = binom.rvs(len(dictionary), .5)
        for x in range(0,Rlen):
            k = random.randint(1, len(dictionary))
            r.append(dictionary.get(k))
        q.append(r)
        r = []
    return(q)


def Random_Resolving_Set_Compliment(unique_words):
    h = Probabilistic_Method2(unique_words)
    #print(h)
   # dictionary = dict.fromkeys( (range(1,len(unique_words))), unique_words)
    #print(dictionary)
    dictionary = dict()
    #print(len(unique_words))
    unique_words_poppy = unique_words
    for i in range(1,len(unique_words_poppy)):
        #dictionary = dict.fromkeys(unique_words.pop(), i)
        dictionary[i] = unique_words_poppy.pop()
    r = []
    q = []
    #print(len(dictionary))
    for i in range(1, h):
        Rlen = binom.rvs(len(dictionary), .5)
        for x in range(0,Rlen):
            k = random.randint(1, len(dictionary))
            r.append(dictionary.get(k))
        q.append(r)
        q.append(list(set(dictionary.values())-set(r)))
        r = []
    return(q)


def JaccardSim(a,b):
    U = a.union(b)
    I = a.intersection(b)
    similarity = (len(I)/len(U))
    return similarity


def JacVector(resolving, test):
    M = []
    for i in range(len(resolving)):
             M.append(JaccardSim(set(resolving[i]),set(test)))
    return(M)

def JVecEMatrix(resolving,tweets):
    veclist = []
    for tweet in tweets:
        veclist.append(JacVector(resolving,tweet))
    #q = scipy.spatial.distance.pdist(veclist, metric='euclidean')
    return veclist

def PCA_Plotting(data, new_dim):
    #planning to add color vector. 
    pca = PCA(n_components=new_dim)
    X_r = pca.fit(data).transform(data)
    colors = ["navy", "turquoise", "darkorange"]
    lw = 2
    X_r=np.transpose(X_r)
    plt.scatter(X_r[0],X_r[1])
    print(pca.explained_variance_ratio_)
    print(pca.n_features_)
    print(pca.n_samples_)
    print(pca.n_components_)

def PCA_Ploting_Dict(data, new_dim, num_tones):
    #planning to allow for 3d. currently works only for 2 tones, 2dim. 
    vec_data = []
    tone_data = []
    for i in range(0,len(data)):
        vec_data.append(data.get(i).get('vec'))
        tone_data.append(data.get(i).get('tone'))
    pca = PCA(n_components=new_dim,)
    X_r = pca.fit(vec_data).transform(vec_data)
    lw = 2
    X_r=np.transpose(X_r)
    plt.figure()
    df = pd.DataFrame(dict(first=X_r[0],second = X_r[1],tone = tone_data))
    print(df)
    colors = {'negative': 'red', 'positive':'blue'}
    plt.scatter(df['first'], df['second'],s=3, c=df['tone'].map(colors))
    print(pca.explained_variance_ratio_)
    print(pca.n_features_)
    print(pca.n_samples_)
    print(pca.n_components_)



In [3]:
def PCA_Ploting_Dict_Tones_axis_tester(data, new_dim, num_tones, axis):
    #planning to allow for 3d. currently works only for , 2dim. 
    #this version takes in which axis you want to plot. requires a new_dim length vector
    print(axis[1])
    vec_data = []
    tone_data = []
    for i in range(0,len(data)):
        vec_data.append(data.get(i).get('vec'))
        tone_data.append(data.get(i).get('tone'))
    pca = PCA(n_components=axis[new_dim - 1],)
    X_r = pca.fit(vec_data).transform(vec_data)
    #colors = ["navy", "turquoise", "darkorange"]
    lw = 2
    X_r=np.transpose(X_r)
    #plt.scatter(X_r[0],X_r[1])
    print(X_r)
    if(new_dim == 2):
        plt.figure()
        plt.xlabel(str(axis[0]))
        plt.ylabel(str(axis[1]))
        colors = ["navy", "turquoise"]
        lw = 2
        df = pd.DataFrame(dict(first=X_r[axis[0]-1],second = X_r[axis[1]-1],tone = tone_data))
        print(df)
        colors = {'negative': 'red', 'positive':'blue', 'neutral': 'brown', 'forth_tone': 'yellow'}
        plt.scatter(df['first'], df['second'], c=df['tone'].map(colors))
        
        print(pca.explained_variance_ratio_)
        print(pca.n_features_)
        print(pca.n_samples_)
        print(pca.n_components_)
    
    elif(new_dim==3):
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        ax.set_xlabel(str(axis[0]))
        ax.set_ylabel(str(axis[1]))
        ax.set_zlabel(str(axis[2]))#labels the axis choice
        colors = ["navy", "turquoise"]
        lw = 13
        df = pd.DataFrame(dict(first=X_r[axis[0]-1],second = X_r[axis[1]-1], third = X_r[axis[2]-1], tone = tone_data))
        print(df)
        colors = {'negative': 'red', 'positive':'blue', 'neutral': 'brown', 'forth_tone': 'yellow'}
        ax.scatter(df['first'], df['second'], df['third'], c=df['tone'].map(colors))

        print(pca.explained_variance_ratio_)
        print(pca.n_features_)
        print(pca.n_samples_)
        print(pca.n_components_)

In [4]:
def k_means_pca_optimizer(data, max_components, num_dimensions, num_clusters, num_tones, tones_list, plotting):
    vec_data = []
    tone_data = []
    score_holder = -2
    score_dummy = -2
    #pca_matrix holder 
    
    if(num_clusters != 0):
        
        for i in range(0,len(data)):
            vec_data.append(data.get(i).get('vec'))
            tone_data.append(data.get(i).get('tone'))


        pca = PCA(n_components=max_components,)
        X_r = pca.fit(vec_data).transform(vec_data)
        
        
        if (num_dimensions==2):
            # need to iterate through to find best PCA axis
            for iterator1 in range(1,max_components):
                for iterator2 in range(1,max_components) :
                    if(iterator1 != iterator2):
                        pca_matrix_dummy = []
                        pca_matrix_dummy = np.column_stack((X_r[:,iterator1],X_r[:,iterator2]))
                        kmeans = KMeans(n_clusters=num_clusters)
                        X_cluster = kmeans.fit_predict(pca_matrix_dummy) # this one predicts the clustering
                        score_dummy = silhouette_score(pca_matrix_dummy,kmeans.labels_)
                        if(score_dummy > score_holder):
                            score_holder = score_dummy
                            pca_matrix_holder = pca_matrix_dummy
                            X_cluster_holder = X_cluster
                            axis = [str(iterator1) +","+str(iterator2)]
                            axis_for_plot = [str(iterator1),str(iterator2)]
                            centers = np.array(kmeans.cluster_centers_)
            
            if (plotting == True):
                #need to figure out how to make this work for arbitrary tones. 
                plt.figure()
                lw = 2
                pca_matrix_holder=np.transpose(pca_matrix_holder)
                df = pd.DataFrame(dict(first = pca_matrix_holder[0],second = pca_matrix_holder[1],tone = tone_data))
                #print(df)
                colors_list = ['red','blue','brown','purple','orange','teal','maroon','fuchsia','grey','tan']
                colors = {}
                dummy_color = 0
                for tone_iterator in tones_list :
                    colors[tone_iterator] = colors_list[dummy_color]
                    dummy_color = dummy_color+1
                plt.scatter(df['first'], df['second'], c=df['tone'].map(colors))
                #centers = np.transpose(centers)
                plt.scatter(centers[:,0], centers[:,1], marker = "x" , color = "black")
                plt.xlabel(axis_for_plot[0])
                plt.ylabel(axis_for_plot[1])
                plt.title("PCA on Tweet Data With")
                #plt.legend()
                plt.show

        elif(num_dimensions == 3):
            for iterator1 in range(1,max_components-1):
                for iterator2 in range(1,max_components-1):
                    for iterator3 in range(1,max_components-1):
                        if(iterator1 != iterator2 and iterator1 != iterator3 and iterator2 != iterator3):
                            pca_matrix_dummy = []
                            kmeans = KMeans(n_clusters=num_clusters)
                            pca_matrix_dummy = np.column_stack((X_r[:,iterator1],X_r[:,iterator2],X_r[:,iterator3]))
                            X_cluster = kmeans.fit_predict(pca_matrix_dummy) # this one predicts the clustering
                            score_dummy = silhouette_score(X_r,kmeans.labels_)

                            if(score_dummy > score_holder):
                                score_holder = score_dummy
                                pca_matrix_holder = pca_matrix_dummy
                                X_cluster_holder = X_cluster
                                axis = [str(iterator1)+ ","+str(iterator2) + "," +str(iterator3)]
                                centers = np.array(kmeans.cluster_centers_)
                                
            if (plotting == True):
                x = list(set(tone_data))
                dic = dict(zip(x, list(range(-1,len(x)-1)))) #this should work with arbitrary tones. 
                tone_data_numeric =[dic[v] for v in tone_data]
                tone_data_numeric = np.asarray(h)
                tone_data_numeric = tone_data_numeric.transpose()
                matlab_matrix = np.column_stack((pca_matrix_holder,tone_data_numeric))
                mdic = {"Data": matlab_matrix, "label": "experiment"}
                filename = r"C:\Users\ajpar\OneDrive\Desktop\Research Documents\ResearchPython\Matlab"  +'\\' + str(max_components) + str(num_dimensions) + str(num_clusters) + str(num_tones) + "function.mat"
                scipy.io.savemat(filename,mdic)
                
                
                
                print("This functionality needs to be improved. Label centers by hand")
                
                #change the positive, negative, neutral to numbers. Then send to matlab using code from evan. 
                #this should simply replace positive negative and neutrals, then open matlab
        else:
            print("This functionality needs to be added")
    
    else:
        print("Currently,call outside loop to find optimal clusters using returning frame")
    returning_frame = pd.DataFrame(dict(score = score_holder, Axis =  axis, Clusters = len(centers)))
    return(returning_frame)

In [ ]:
#no mix

#Devoted to importing and getting wordlist using vader lexicon
#https://github.com/cjhutto/vaderSentiment#code-examples

# note, I'm not bothering to do stemming and lemmatizing on this stuff, since in terms of a toy example it shouldnt matter
#if this is to be used in the future, will have to write code to deal with words w/ same token (hope vs hopeful)
#being in different lists. Could result in conflict. 
n = 350
imported_vader_words = []
negative_vader_list = []
neutral_vader_list = []
positive_vader_list = []
lam_vary = {}
lam_vary_prime = {}
x = open(r"C:\Users\ajpar\OneDrive\Desktop\Research Documents\Project Coding\Sample_Word_Datasets\Full_Vader_Senti.txt", "r")
#e = r"C:\Users\ajpar\Desktop\Research Documents\Project Coding\Full_Tweet_List_Processed.txt"
for line in x:
    
    l = line.strip('\n')
    l = l.split("\t")
    imported_vader_words.append(l)
    
x.close()

for y in imported_vader_words :
    if (-1<= float(y[1]) <= 1):
        neutral_vader_list.append(y[0]) 
    elif (float(y[1]) < -1):
        negative_vader_list.append(y[0])
    else :
        positive_vader_list.append(y[0])


shorten_pos = random.sample(positive_vader_list, n) #gets random n words w/out duplicates
shorten_neg = random.sample(negative_vader_list, n) #gets random n words w/out duplicates
shorten_neutral = random.sample(neutral_vader_list, n) #gets random n words w/out duplicates

for lam in range(5,100,5):
    negative_sent_list = []
    for i in range(0,500):
        h = np.random.poisson(lam=lam, size=None)
        negative_sent_list.append(random.sample(shorten_neg, h))

    positive_sent_list = []
    for i in range(0,500):
        h = np.random.poisson(lam=lam, size=None)
        positive_sent_list.append(random.sample(shorten_pos, h))
    #print(negative_sent_list)

    #print(positive_sent_list)

    neutral_sent_list = []

    for i in range(0,500):
        h = np.random.poisson(lam=lam, size=None)
        neutral_sent_list.append(random.sample(shorten_neutral, h))

    final_list = negative_sent_list + positive_sent_list + neutral_sent_list


    test_words2 = {x for l in final_list for x in l}
    test_words2_prime = {x for l in final_list for x in l}
    resolve_test2 = Random_Resolving_Set(test_words2)
    resolve_test2_prime = Random_Resolving_Set_Compliment(test_words2_prime)
    print(len(resolve_test2))

    x_test2 = JVecEMatrix(resolve_test2,final_list)
    x_test2_prime = JVecEMatrix(resolve_test2_prime,final_list)
    #print(x_test)

    vec_dic_test7 = {}
    vec_dic_test7_prime = {}
    for i in range(0,len(x_test2)):
        if (i<500):
            vec_dic_test7[i] = {'vec': x_test2[i], 'tone': "negative"}
        elif (500<i<1000):
            vec_dic_test7[i] = {'vec': x_test2[i], 'tone': "positive"}
        else:
            vec_dic_test7[i] = {'vec': x_test2[i], 'tone': "neutral"}
        
    lam_vary[lam] = vec_dic_test7
    for i in range(0,len(x_test2_prime)):
            if (i<500):
                vec_dic_test7_prime[i] = {'vec': x_test2_prime[i], 'tone': "negative"}
            elif (500<i<1000):
                vec_dic_test7_prime[i] = {'vec': x_test2_prime[i], 'tone': "positive"}
            else:
                vec_dic_test7_prime[i] = {'vec': x_test2_prime[i], 'tone': "neutral"}
    lam_vary_prime[lam] = vec_dic_test7_prime